In [1]:
import json

import pandas as pd

###################################

from datafin.apis import PolygonClient              #type: ignore

from datafin.aws import SecretsClient              #type: ignore
from datafin.aws import S3Client                   #type: ignore

from datafin.utils import GmailClient              #type: ignore
from datafin.utils import (                                #type: ignore
    now,
    to_ny_time,
    string_formating,
    get_ny_timestamp_for_today_time_range,
    is_today_a_trading_day
)   

secrets = SecretsClient()

In [2]:
s3 = S3Client(
    aws_access_key_id = secrets.aws_access_key,
    aws_secret_access_key = secrets.aws_secret_access_key,
    bucket_name = secrets.get_bucket_name()
)

In [3]:
# @op
def portfolio_list(
        # secrets: SecretsResource
) -> list:
    
    s3 = S3Client(
        # bucket_name = secrets.client.get_bucket_name(),
        # aws_access_key_id = secrets.client.aws_access_key,
        # aws_secret_access_key = secrets.client.aws_secret_access_key,
        bucket_name = secrets.get_bucket_name(),
        aws_access_key_id = secrets.aws_access_key,
        aws_secret_access_key = secrets.aws_secret_access_key,
        region_name = 'us-east-1'
    )

    raw_json_list = s3.get_json(
        path = 'v1/reference',
        file_name = 'portfolio'
    )

    cleaned_list = list(raw_json_list['data'])
    return cleaned_list

In [4]:
# @op
def portfolio_combined_df(
        input_list
) -> None:
    
    pg = PolygonClient(
        # api_key = secrets.client.get_polygon_api_key(),
        api_key = secrets.get_polygon_api_key()
    )

    timestamps = get_ny_timestamp_for_today_time_range(
        _from = (9, 30, 0),
        _to = (16, 0, 0)
    )

    df_start = pd.DataFrame()

    for symbol in input_list:
        aggs = pg.get_aggs(
            symbol = symbol,
            multiplier = 15,
            unit = 'minute',
            _from = timestamps[0],
            _to = timestamps[1]
        )
        df = pd.DataFrame(aggs)
        df['symbol'] = symbol
        df_start = pd.concat([df_start, df], ignore_index=True)
    return df_start


portfolio_combined_df(portfolio_list())

,open,high,low,close,volume,vwap,timestamp,transactions,otc,symbol
0,215.195,216.6200,215.00,216.320,2523336.0,215.8012,1750167000000,39012,None,AMZN
1,216.330,217.3600,216.20,217.280,2022297.0,216.8600,1750167900000,31582,None,AMZN
2,217.290,217.4100,216.12,216.120,1408764.0,216.8072,1750168800000,23141,None,AMZN
3,216.110,216.7100,215.95,215.980,1833974.0,216.3578,1750169700000,19030,None,AMZN
4,215.980,216.2700,215.53,215.925,862482.0,215.9283,1750170600000,14410,None,AMZN
...,...,...,...,...,...,...,...,...,...,...
130,529.560,529.6700,528.81,529.340,1104305.0,529.2623,1750186800000,15748,None,QQQ
131,529.320,529.8400,529.12,529.760,956716.0,529.5877,1750187700000,13529,None,QQQ
132,529.780,529.9591,529.18,529.535,1387266.0,529.6141,1750188600000,20249,None,QQQ
133,529.540,530.0800,528.78,529.000,3685234.0,529.3135,1750189500000,47789,None,QQQ


In [5]:
pg = PolygonClient(
    # api_key = secrets.client.get_polygon_api_key(),
    api_key = secrets.get_polygon_api_key()
)

pg.get_previous_close_agg("amzn")

[PreviousCloseAgg(ticker='AMZN', close=214.82, high=217.41, low=214.56, open=215.195, timestamp=1750190400000, volume=32115161.0, vwap=215.7324)]